In [1]:
import retro
import os
import cv2
import numpy as np
import optuna
from gymnasium import Env
from gymnasium.spaces import MultiBinary, Box
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback  
import time

In [2]:
class StreetFighter(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        self.game = retro.make(
            game="StreetFighterIISpecialChampionEdition-Genesis",
            use_restricted_actions=retro.Actions.FILTERED
        )
        self.previous_frame = np.zeros((84, 84, 1), dtype=np.uint8)
        self.score = 0

    def preprocess(self, observation):
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_CUBIC)
        return np.reshape(resized, (84, 84, 1))

    def reset(self, **kwargs):
        obs = self.game.reset()
        obs = self.preprocess(obs)
        self.previous_frame = obs
        self.score = 0
        return obs, {}

    def step(self, action):
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs
        
        reward = info.get('score', 0) - self.score
        self.score = info.get('score', 0)
        
        terminated = done
        truncated = False
        return frame_delta, reward, terminated, truncated, info

    def render(self, *args, **kwargs):
        self.game.render(*args, **kwargs)

    def close(self):
        self.game.close()

In [3]:
LOG_DIR = "./logs/"
OPT_DIR = "./opt/"
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(OPT_DIR, exist_ok=True)

In [9]:
def optimize_ppo(trial):
    return {
        'n_steps': trial.suggest_int('n_steps', 2048, 8192),
        'gamma': trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range': trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda': trial.suggest_uniform('gae_lambda', 0.8, 0.99),
         
    }

def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial)
        
        print(f"\nStarting Trial {trial.number}")
        print("Parameters:")
        for key, value in model_params.items():
            print(f"  {key}: {value}")

        env = StreetFighter()
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, n_stack=4, channels_order='last')

        model = PPO(
            'CnnPolicy',
            env,
            verbose=0,
            tensorboard_log=LOG_DIR,
            **model_params
        )

        model.learn(total_timesteps=100000)

        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, f"trial_{trial.number}_best_model")
        model.save(SAVE_PATH)

        print(f"\nTrial {trial.number} finished")
        print(f"Mean reward: {mean_reward}")
        print("Parameters:")
        for key, value in model_params.items():
            print(f"  {key}: {value}")

        return mean_reward

    except Exception as e:
        print(f"\nTrial failed due to: {e}")
        return -1000

def print_best_trial(study, trial):
    print(f"\nCurrent best trial: {study.best_trial.number}")
    print(f"Best value: {study.best_trial.value}")
    print("Best params:")
    for key, value in study.best_trial.params.items():
        print(f"  {key}: {value}")

# Run optimization
study = optuna.create_study(direction='maximize')
study.optimize(
    optimize_agent, 
    n_trials=1000, 
    n_jobs=1,
    callbacks=[print_best_trial]
)

# Save the best model after all trials
if study.best_trial:
    best_model_path = os.path.join(OPT_DIR, "best_model")
    best_model = PPO.load(os.path.join(OPT_DIR, f"trial_{study.best_trial.number}_best_model"))
    best_model.save(best_model_path)
    print(f"\nSaved best model from trial {study.best_trial.number} to {best_model_path}")

[I 2025-04-08 14:23:52,264] A new study created in memory with name: no-name-2854ce20-1257-40c0-a632-f1b89490df18
C:\Users\ajogd\AppData\Local\Temp\ipykernel_43712\142763229.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial.suggest_loguniform('gamma', 0.8, 0.9999),
C:\Users\ajogd\AppData\Local\Temp\ipykernel_43712\142763229.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
C:\Users\ajogd\AppData\Local\Temp\ipykernel_43712\142763229.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releas


Starting Trial 0
Parameters:
  n_steps: 2234
  gamma: 0.9678535745643071
  learning_rate: 1.5818461356559774e-05
  clip_range: 0.183758152402001
  gae_lambda: 0.8889790303772501


c:\users\ajogd\onedrive\desktop\new folder (4)\venv\lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2234`, after every 34 untruncated mini-batches, there will be a truncated mini-batch of size 58
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2234 and n_envs=1)
  warnings.warn(
[I 2025-04-08 14:34:30,492] Trial 0 finished with value: 5000.0 and parameters: {'n_steps': 2234, 'gamma': 0.9678535745643071, 'learning_rate': 1.5818461356559774e-05, 'clip_range': 0.183758152402001, 'gae_lambda': 0.8889790303772501}. Best is trial 0 with value: 5000.0.



Trial 0 finished
Mean reward: 5000.0
Parameters:
  n_steps: 2234
  gamma: 0.9678535745643071
  learning_rate: 1.5818461356559774e-05
  clip_range: 0.183758152402001
  gae_lambda: 0.8889790303772501

Current best trial: 0
Best value: 5000.0
Best params:
  n_steps: 2234
  gamma: 0.9678535745643071
  learning_rate: 1.5818461356559774e-05
  clip_range: 0.183758152402001
  gae_lambda: 0.8889790303772501

Starting Trial 1
Parameters:
  n_steps: 6808
  gamma: 0.9548798932936535
  learning_rate: 1.9484615633222276e-05
  clip_range: 0.32094729568276137
  gae_lambda: 0.8498948429129224


c:\users\ajogd\onedrive\desktop\new folder (4)\venv\lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 6808`, after every 106 untruncated mini-batches, there will be a truncated mini-batch of size 24
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=6808 and n_envs=1)
  warnings.warn(
[I 2025-04-08 14:46:36,150] Trial 1 finished with value: 1000.0 and parameters: {'n_steps': 6808, 'gamma': 0.9548798932936535, 'learning_rate': 1.9484615633222276e-05, 'clip_range': 0.32094729568276137, 'gae_lambda': 0.8498948429129224}. Best is trial 0 with value: 5000.0.


...


Trial 7 finished
Mean reward: 1500.0
Parameters:
  n_steps: 5919
  gamma: 0.8754413897158089
  learning_rate: 3.989971379684793e-05
  clip_range: 0.12406691266492693
  gae_lambda: 0.925169234690562

Current best trial: 3
Best value: 45600.0
Best params:
  n_steps: 3795
  gamma: 0.9003445466748287
  learning_rate: 4.960542238487099e-05
  clip_range: 0.23687395041796724
  gae_lambda: 0.8192653363600547

Starting Trial 8
Parameters:
  n_steps: 2504
  gamma: 0.8248434861766549
  learning_rate: 1.7645720546779833e-05
  clip_range: 0.11598136779051013
  gae_lambda: 0.8641566910901479


c:\users\ajogd\onedrive\desktop\new folder (4)\venv\lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2504`, after every 39 untruncated mini-batches, there will be a truncated mini-batch of size 8
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2504 and n_envs=1)
  warnings.warn(
[I 2025-04-08 16:05:09,195] Trial 8 finished with value: 4200.0 and parameters: {'n_steps': 2504, 'gamma': 0.8248434861766549, 'learning_rate': 1.7645720546779833e-05, 'clip_range': 0.11598136779051013, 'gae_lambda': 0.8641566910901479}. Best is trial 3 with value: 45600.0.



Trial 8 finished
Mean reward: 4200.0
Parameters:
  n_steps: 2504
  gamma: 0.8248434861766549
  learning_rate: 1.7645720546779833e-05
  clip_range: 0.11598136779051013
  gae_lambda: 0.8641566910901479

Current best trial: 3
Best value: 45600.0
Best params:
  n_steps: 3795
  gamma: 0.9003445466748287
  learning_rate: 4.960542238487099e-05
  clip_range: 0.23687395041796724
  gae_lambda: 0.8192653363600547

Starting Trial 9
Parameters:
  n_steps: 5230
  gamma: 0.8147419319162601
  learning_rate: 3.7924326880121175e-05
  clip_range: 0.2916564660522714
  gae_lambda: 0.9768708220445755


c:\users\ajogd\onedrive\desktop\new folder (4)\venv\lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 5230`, after every 81 untruncated mini-batches, there will be a truncated mini-batch of size 46
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=5230 and n_envs=1)
  warnings.warn(
[I 2025-04-08 16:17:39,887] Trial 9 finished with value: 0.0 and parameters: {'n_steps': 5230, 'gamma': 0.8147419319162601, 'learning_rate': 3.7924326880121175e-05, 'clip_range': 0.2916564660522714, 'gae_lambda': 0.9768708220445755}. Best is trial 3 with value: 45600.0.



Trial 9 finished
Mean reward: 0.0
Parameters:
  n_steps: 5230
  gamma: 0.8147419319162601
  learning_rate: 3.7924326880121175e-05
  clip_range: 0.2916564660522714
  gae_lambda: 0.9768708220445755

Current best trial: 3
Best value: 45600.0
Best params:
  n_steps: 3795
  gamma: 0.9003445466748287
  learning_rate: 4.960542238487099e-05
  clip_range: 0.23687395041796724
  gae_lambda: 0.8192653363600547

Saved best model from trial 3 to ./opt/best_model


In [4]:
model = PPO.load(os.path.join(OPT_DIR, 'trial_3_best_model.zip'))

In [39]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self,check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def __init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok = True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model{}'.format(self.n_calls))
            self.model.save(model_path)
        return True

In [40]:
CHECKPOINT_DIR = './train/'

In [41]:
model_params = params={'n_steps': 3776, 'gamma': 0.9003445466748287, 'learning_rate': 4.960542238487099e-05, 'clip_range': 0.23687395041796724, 'gae_lambda': 0.8192653363600547}

In [43]:
env.close()

In [44]:
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4, channels_order='last')

In [45]:
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, **model_params)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [47]:
model.load(os.path.join(OPT_DIR, 'trial_3_best_model.zip'))

In [50]:

callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)


In [48]:
model.learn(total_timesteps=100000, callback = callback)

NameError: name 'callback' is not defined

In [52]:
model = PPO.load("./opt/trial_3_best_model.zip")

In [53]:
env.close()

In [55]:
mean_reward , _ = evaluate_policy(model, env, render=True, n_eval_episodes=1)

AttributeError: 'RetroEnv' object has no attribute 'em'

In [36]:
mean_reward

2300.0

In [24]:
obs,_= env.reset()

AttributeError: 'RetroEnv' object has no attribute 'em'

In [ ]:
# Close existing environment
env.close()

# Recreate the environment
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4, channels_order='last')

# Start fresh
obs = env.reset()
done = False
for game in range(1):
    while not done:
        env.envs[0].render()

        action, _ = model.predict(obs)  # <-- fixed line
        obs, reward, done, info = env.step(action)
        time.sleep(0.01)
        if reward > 0:
            print(reward)



[500.]
[400.]
[500.]
[1000.]
[500.]
[500.]
